In [1]:
questions_on_empiricism = [
    'Alex can see things with his eyes. When could Alex see with his eyes for the first time?',
    'When there is a sound close by, Alex can hear it. When could Alex hear sounds for the first time?',
    'When seeing a red flower and a blue flower, Alex can tell that they are different colors. Alex can tell colors apart. \
    When could Alex tell colors apart for the first time?',
    'When there is a car approaching, Alex can tell that the car is getting closer. Alex can tell what is near and what is far. \
    When could Alex tell near and far for the first time?',
    'When Alex sees someone hold an object and then drop it, Alex thinks the object will fall. Alex thinks objects will fall if we let go of them.\
     When could Alex think that for the first time?',
    'If Alex sees a toy being hidden in a box, he will think the object is still there even though he can no longer see it.\
     When could Alex think that for the first time?',
    'If Alex sees two cookies, one with 5 chocolate chips in it and one with 20 chocolate chips in it, he can tell which cookie has more chocolate chips without counting. \
     When could Alex tell which has more for the first time?',
    'If Alex sees a turtle that is upside down and struggling to get on its feet, he thinks that he should help the turtle. Alex thinks that helping is the right thing to do. \
    When could Alex think that for the first time?',
    'Alex can read books. When could Alex read for the first time? '
]

In [2]:
import kagglehub
import glob

path = kagglehub.dataset_download("aiolapo/fgnet-dataset")

age_paths = {'newborn': glob.glob(f"{path}/**/080A00.JPG", recursive=True)[0], "older_infant": glob.glob(f"{path}/**/080A01.JPG", recursive=True)[0],
            'toddler': glob.glob(f"{path}/**/080A02.JPG", recursive=True)[0], 'preschool_child': glob.glob(f"{path}/**/080A04.JPG", recursive=True)[0],
            'schoolage_child': glob.glob(f"{path}/**/080A07.JPG", recursive=True)[0]}

C:\Users\yh1242\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import huggingface_hub
huggingface_hub.login('')

In [ ]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch
import random
random.seed(42)

model_id = "google/gemma-3-27b-it"
model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a human answering questions for a psychology survey."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "url": age_paths['newborn']},
          {"type": "image", "url": age_paths['toddler']},
          {"type": "image", "url": age_paths['schoolage_child']},
          {"type": "text", "text": f'{questions_on_empiricism[0]} Pick from image 1, 2, 3. \
           You must reply with either 1, 2, or 3 and specify the age of the child.'},
        ]
    }
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)


C:\Users\yh1242\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yh1242\.cache\huggingface\hub\models--google--gemma-3-27b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shard

1.

Babies are born with some vision, but it's quite blurry at first. Vision develops rapidly in the first few months. Image 1 shows a very young baby, likely around 3-6 months old, when vision is starting to become more focused and they can begin to track objects and recognize faces. Images 2 and 3 show older children with more developed vision, but the *first* time Alex could see would have been as a baby in image 1.


In [4]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch

model_id = "google/gemma-3-27b-it"
model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a human answering questions for a psychology survey."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "url": age_paths['newborn']},
          {"type": "image", "url": age_paths['toddler']},
          {"type": "image", "url": age_paths['schoolage_child']},
          {"type": "text", "text": f'{questions_on_empiricism[-2]} Pick from image 1, 2, 3. \
           You must reply with either 1, 2, or 3 and specify the age of the child.'},
        ]
    }
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Loading checkpoint shards: 100%|██████████| 12/12 [00:22<00:00,  1.88s/it]
Some parameters are on the meta device because they were offloaded to the disk and cpu.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


3 – Approximately 6-7 years old. 

Here's my reasoning:

The capacity for empathy and understanding the distress of another being, and then acting on a moral imperative to help, develops significantly around 6-7 years old. While babies (image 1) and toddlers (image 2) can show *some* emotional response, it's not the same as consciously understanding the turtle's plight and feeling a sense of obligation to assist. By age 6


### use a smaller version of gemma

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline("text-generation", model="google/gemma-3-1b-it", device="cuda", torch_dtype=torch.bfloat16)
messages = [
    [
        {
            "role": "system",
             "content": [{"type": "text", "text": "You are a human answering questions for a psychology survey."}]
        },
        {
            "role": "user",
           "content": [
            {"type": "image", "url": age_paths['newborn']},
          {"type": "image", "url": age_paths['toddler']},
          {"type": "image", "url": age_paths['schoolage_child']},
          {"type": "text", "text": f'{questions_on_empiricism[0]} Pick from image 1, 2, 3. \
           You must reply with either 1, 2, or 3 and specify the age of the child.'},
        ]
        },
    ],
]

output = pipe(messages, max_new_tokens=50)
output

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda


[[{'generated_text': [{'role': 'system',
     'content': [{'type': 'text',
       'text': 'You are a human answering questions for a psychology survey.'}]},
    {'role': 'user',
     'content': [{'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A00.JPG'},
      {'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A02.JPG'},
      {'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A07.JPG'},
      {'type': 'text',
       'text': 'Alex can see things with his eyes. When could Alex see with his eyes for the first time? Pick from image 1, 2, 3.            You must reply with either 1, 2, or 3 and specify the age of the child.'}]},
    {'role': 'assistant',
     'content': "Okay, let's analyze this.\n\nLooking at the images, **image 1** is t

In [ ]:
from transformers import pipeline
import torch

pipe = pipeline("text-generation", model="google/gemma-3-1b-it", device="cuda", torch_dtype=torch.bfloat16)
messages = [
    [
        {
            "role": "system",
             "content": [{"type": "text", "text": "You are a human answering questions for a psychology survey."}]
        },
        {
            "role": "user",
           "content": [
            {"type": "image", "url": age_paths['newborn']},
          {"type": "image", "url": age_paths['toddler']},
          {"type": "image", "url": age_paths['schoolage_child']},
          {"type": "text", "text": f'{questions_on_empiricism[-2]} Pick from image 1, 2, 3. \
           You must reply with either 1, 2, or 3 and specify the age of the child.'},
        ]
        },
    ],
]

output = pipe(messages, max_new_tokens=50)
output

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda


[[{'generated_text': [{'role': 'system',
     'content': [{'type': 'text',
       'text': 'You are a human answering questions for a psychology survey.'}]},
    {'role': 'user',
     'content': [{'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A00.JPG'},
      {'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A02.JPG'},
      {'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A07.JPG'},
      {'type': 'text',
       'text': 'If Alex sees a turtle that is upside down and struggling to get on its feet, he thinks that he should help the turtle. Alex thinks that helping is the right thing to do.     When could Alex think that for the first time? Pick from image 1, 2, 3.            You must reply with either 1, 2, or 3 and specify the ag

In [4]:
from transformers import pipeline
import torch

pipe = pipeline("text-generation", model="google/gemma-3-1b-it", device="cuda", torch_dtype=torch.bfloat16)
messages = [
    [
        {
            "role": "system",
             "content": [{"type": "text", "text": "You are a human answering questions for a psychology survey."}]
        },
        {
            "role": "user",
           "content": [
            {"type": "image", "url": age_paths['newborn']},
          {"type": "image", "url": age_paths['toddler']},
          {"type": "image", "url": age_paths['schoolage_child']},
          {"type": "text", "text": f'{questions_on_empiricism[6]} Pick from image 1, 2, 3. \
           You must reply with either 1, 2, or 3 and specify the age of the child.'},
        ]
        },
    ],
]

output = pipe(messages, max_new_tokens=50)
output

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda


[[{'generated_text': [{'role': 'system',
     'content': [{'type': 'text',
       'text': 'You are a human answering questions for a psychology survey.'}]},
    {'role': 'user',
     'content': [{'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A00.JPG'},
      {'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A02.JPG'},
      {'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A07.JPG'},
      {'type': 'text',
       'text': 'If Alex sees two cookies, one with 5 chocolate chips in it and one with 20 chocolate chips in it, he can tell which cookie has more chocolate chips without counting.      When could Alex tell which has more for the first time? Pick from image 1, 2, 3.            You must reply with either 1, 2, or 3 and specify 

In [4]:
from transformers import pipeline
import torch

pipe = pipeline("text-generation", model="google/gemma-3-1b-it", device="cuda", torch_dtype=torch.bfloat16)
messages = [
    [
        {
            "role": "system",
             "content": [{"type": "text", "text": "You are a human answering questions for a psychology survey."}]
        },
        {
            "role": "user",
           "content": [
            {"type": "image", "url": age_paths['newborn']},
          {"type": "image", "url": age_paths['toddler']},
          {"type": "image", "url": age_paths['schoolage_child']},
          {"type": "text", "text": f'{questions_on_empiricism[1]} Pick from image 1, 2, 3. \
           You must reply with either 1, 2, or 3 and specify the age of the child.'},
        ]
        },
    ],
]

output = pipe(messages, max_new_tokens=50)
output

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda


[[{'generated_text': [{'role': 'system',
     'content': [{'type': 'text',
       'text': 'You are a human answering questions for a psychology survey.'}]},
    {'role': 'user',
     'content': [{'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A00.JPG'},
      {'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A02.JPG'},
      {'type': 'image',
       'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A07.JPG'},
      {'type': 'text',
       'text': 'When there is a sound close by, Alex can hear it. When could Alex hear sounds for the first time? Pick from image 1, 2, 3.            You must reply with either 1, 2, or 3 and specify the age of the child.'}]},
    {'role': 'assistant', 'content': '2\n\nAge 8'}]}]]

In [4]:
from transformers import pipeline
import torch
import random
random.seed(42)

pipe = pipeline("text-generation", model="google/gemma-3-1b-it", device="cuda", torch_dtype=torch.bfloat16)
for i in range(len(questions_on_empiricism)):
    messages = [
        [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are a human answering questions for a psychology survey."}]
            },
            {
                "role": "user",
            "content": [
                {"type": "image", "url": age_paths['newborn']},
            {"type": "image", "url": age_paths['toddler']},
            {"type": "image", "url": age_paths['schoolage_child']},
            {"type": "text", "text": f'{questions_on_empiricism[i]} Pick from image 1, 2, 3. \
            You must reply with either 1, 2, or 3 and specify the age of the child.'},
            ]
            },
        ],
    ]

    output = pipe(messages, max_new_tokens=50)
    print(f'{questions_on_empiricism[i]}')
    print(output)
    print('\n')

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda


Alex can see things with his eyes. When could Alex see with his eyes for the first time?
[[{'generated_text': [{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a human answering questions for a psychology survey.'}]}, {'role': 'user', 'content': [{'type': 'image', 'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A00.JPG'}, {'type': 'image', 'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A02.JPG'}, {'type': 'image', 'url': 'C:\\Users\\yh1242\\.cache\\kagglehub\\datasets\\aiolapo\\fgnet-dataset\\versions\\1\\FGNET\\images\\080A07.JPG'}, {'type': 'text', 'text': 'Alex can see things with his eyes. When could Alex see with his eyes for the first time? Pick from image 1, 2, 3.             You must reply with either 1, 2, or 3 and specify the age of the child.'}]}, {'role': 'assistant', 'content': 'Okay, let’s analyze this.\n\nBased on the image, Alex 

### Previous version of Gemma (gemma 2)
> https://huggingface.co/google/gemma-2-2b-it

In [4]:
from transformers import pipeline
import torch
import random
random.seed(42)

pipe = pipeline("text-generation", model="google/gemma-2-2b-it", device="cuda", torch_dtype=torch.bfloat16)

for i in range(len(questions_on_empiricism)):
    messages = [
        {"role": "user", "content":  f'{questions_on_empiricism[i]} Please specify ONE age.'}
    ]

    outputs = pipe(messages, max_new_tokens=50)
    assistant_response = outputs[0]["generated_text"][-1]["content"].strip()
    print(f'{questions_on_empiricism[i]}')
    print(assistant_response)
    print('\n')

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 84.56it/s]
Device set to use cuda


Alex can see things with his eyes. When could Alex see with his eyes for the first time?
Alex could see with his eyes for the first time around **newborn**, or **around 0-2 months old**.


When there is a sound close by, Alex can hear it. When could Alex hear sounds for the first time?
The answer is **around 6 to 8 months old**. 

Here's why: 

* **Infancy:**  Babies start to develop a sense of hearing around 6 months old.  They begin to distinguish between different sounds


When seeing a red flower and a blue flower, Alex can tell that they are different colors. Alex can tell colors apart.     When could Alex tell colors apart for the first time?
Alex could tell colors apart for the first time around **6 months old**. 

Here's why:

* **Around 6 months**, babies begin to show a growing ability to discriminate colors. 
* This development is part of their


When there is a car approaching, Alex can tell that the car is getting closer. Alex can tell what is near and what is far.     Whe